In [45]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_rows', 100)

### Load and explore the data

In [46]:
test_data = pd.read_csv("../data/titanic/test.csv")

print(test_data.info())
print('\n')
print(test_data.isna().sum())
print('\n')
test_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64




,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [47]:
train_data = pd.read_csv("../data/titanic/train.csv")

print(train_data.info())
print('\n')
print(train_data.isna().sum())
print('\n')
train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: in

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [48]:
gender_submission_data = pd.read_csv("../data/titanic/gender_submission.csv")

print(gender_submission_data.info())
print('\n')
print(gender_submission_data.isna().sum())
print('\n')
gender_submission_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB
None


PassengerId    0
Survived       0
dtype: int64




,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


### Check how many people survived wrt gender

In [49]:
train_data.groupby(['Sex','Survived']).agg(count = ('Survived','count'))

count
Sex    Survived       
female 0            81
       1           233
male   0           468
       1           109

### Try fitting a dummy model based on gender to predict survival

In [51]:
def my_dummy_model(sex): 
    if sex == 'female': 
        return 1
    elif sex == 'male':
        return 0

train_data['preds'] = [my_dummy_model(sex) for sex in train_data['Sex']]
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,preds
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [52]:
error_check = train_data.filter(['Survived','preds']).assign(check = lambda x: x['Survived'] == x['preds'])
error_check.sample(10)

,Survived,preds,check
138,0,0,True
782,0,0,True
383,1,1,True
322,1,1,True
487,0,0,True
624,0,0,True
330,1,1,True
416,1,1,True
34,0,0,True
132,0,1,False


In [53]:
# total number of correct answers
error_check['check'].sum()

701

In [54]:
# accuracy of our model is: 
error_check['check'].sum() / error_check.shape[0]

0.7867564534231201

In [55]:
train_data.groupby(['Survived']).agg(n=('Survived','count')).assign(perc = lambda x: x['n']/x['n'].sum())

,n,perc
Survived,,
0,549,0.616162
1,342,0.383838


###  Split the data into train and test sets

In [66]:
X = train_data.filter(['Sex'])
y = train_data.filter(['Survived'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=8)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

### One-hot encoding

In [67]:
# Change sex column to one-hot encoded data with pandas
pd.get_dummies(X_train).head()

,Sex_female,Sex_male
430,0,1
66,1,0
39,1,0
589,0,1
218,1,0


In [68]:
# Change sex column to one-hot encoded data with sklearn
enc = OneHotEncoder(drop='first', sparse=False) # handle_unknown='ignore'
enc.fit(X_train)

OneHotEncoder(drop='first', sparse=False)

In [69]:
# printing the category names

# as we know we have specified `drop='first'`, we are going only to keep
# the category male, which is in the 2nd positions of the array
enc.get_feature_names().tolist()

['x0_male']

In [71]:
X_train_enc = enc.transform(X_train)
X_train_enc = pd.DataFrame(X_train_enc, columns=enc.get_feature_names().tolist())
X_train_enc.head()

,x0_male
0,1.0
1,0.0
2,0.0
3,1.0
4,0.0


### Try out a dummy model on the full dataset

In [91]:
X = train_data.drop(columns=['Survived'])
y = train_data.filter(['Survived'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=8)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,preds
430,431,1,"Bjornstrom-Steffansson, Mr. Mauritz Hakan",male,28.0,0,0,110564,26.5500,C52,S,0
66,67,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.0,0,0,C.A. 29395,10.5000,F33,S,1
39,40,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C,1
589,590,3,"Murdlin, Mr. Joseph",male,NaN,0,0,A./5. 3235,8.0500,NaN,S,0
218,219,1,"Bazzani, Miss. Albina",female,32.0,0,0,11813,76.2917,D15,C,1
...,...,...,...,...,...,...,...,...,...,...,...,...
136,137,1,"Newsom, Miss. Helen Monypeny",female,19.0,0,2,11752,26.2833,D47,S,1
133,134,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.0000,NaN,S,1
361,362,2,"del Carlo, Mr. Sebastiano",male,29.0,1,0,SC/PARIS 2167,27.7208,NaN,C,0
340,341,2,"Navratil, Master. Edmond Roger",male,2.0,1,1,230080,26.0000,F2,S,0


In [92]:
def my_dummy_model(row): 
    
    if row.Sex == 'female': 
        if row.Pclass == 1:
            return 1
        
        if row.Pclass == 2:
            return 1
        
        if row.Pclass == 3:
            return 0
        
    elif row.Sex == 'male':
        if row.Pclass == 1:
            return 1
        
        if row.Pclass == 2:
            return 0
        
        if row.Pclass == 3:
            return 0

train_data['preds'] = [my_dummy_model(row) for row in train_data.iterrows()]
train_data.head()
'''
error_check = train_data.filter(['Survived','preds']).assign(check = lambda x: x['Survived'] == x['preds'])

# accuracy of our model is: 
error_check['check'].sum() / error_check.shape[0]'''

TypeError: tuple indices must be integers or slices, not str

In [73]:
enc = OneHotEncoder(drop='first', sparse=False) # handle_unknown='ignore'
enc.fit(X_train)
enc.transform(X_train.Sex)

#X_train_enc = pd.DataFrame(X_train_enc, columns=enc.get_feature_names().tolist())
#X_train_enc.head()

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

X_object = X.select_dtypes('object')
ohe.fit(X_object)

codes = ohe.transform(X_object).toarray()
feature_names = ohe.get_feature_names(['string1', 'string2'])

X = pd.concat([df.select_dtypes(exclude='object'), 
               pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)

ValueError: Expected 2D array, got 1D array instead:
array=['male' 'female' 'female' 'male' 'female' 'male' 'male' 'female' 'male'
 'male' 'female' 'female' 'male' 'male' 'male' 'male' 'male' 'female'
 'male' 'male' 'male' 'male' 'male' 'male' 'female' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'female' 'male' 'female' 'male' 'female'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'female'
 'female' 'male' 'male' 'male' 'male' 'male' 'female' 'male' 'male' 'male'
 'male' 'male' 'male' 'female' 'male' 'male' 'male' 'female' 'male' 'male'
 'male' 'female' 'male' 'male' 'male' 'female' 'female' 'male' 'male'
 'female' 'male' 'female' 'male' 'male' 'female' 'female' 'male' 'female'
 'female' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'female' 'male' 'male'
 'male' 'male' 'female' 'male' 'male' 'male' 'male' 'female' 'male'
 'female' 'male' 'female' 'male' 'female' 'male' 'male' 'male' 'male'
 'fem

In [83]:
train_data.groupby(['Sex','Survived']).agg(count = ('Survived','count'))

count
Sex    Survived       
female 0            81
       1           233
male   0           468
       1           109

In [84]:
train_data.groupby(['Pclass','Survived']).agg(count = ('Survived','count'))

count
Pclass Survived       
1      0            80
       1           136
2      0            97
       1            87
3      0           372
       1           119

In [85]:
train_data.groupby(['Pclass','Survived']).agg(count = ('Survived','count')).assign(perc = lambda x: x['count']/x['count'].sum())

count      perc
Pclass Survived                 
1      0            80  0.089787
       1           136  0.152637
2      0            97  0.108866
       1            87  0.097643
3      0           372  0.417508
       1           119  0.133558

In [81]:
train_data.groupby(['Pclass']).agg(n=('Survived','count')).assign(perc = lambda x: x['n']/x['n'].sum())

,n,perc
Pclass,,
1,216,0.242424
2,184,0.206510
3,491,0.551066
